# 0. Loading data

In [2]:
import pandas as pd

## 0.1. Firstly we load the data

In [5]:
df = pd.read_csv("/home/juls/Escritorio/T2.csv")
list(df)
print(df.shape)

(78646, 245)


/home/juls/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 1. Filtering

## 1.1. Filter rows

The first thing we need to do is to convert strings to datetime since we'll filter rows by date.

In [6]:
df['TimeStemp'] = pd.to_datetime(df['TimeStemp'])

Then we can apply a filter by date.

In [30]:
df_3d = df[(df['TimeStemp'] > '2016-04-28 00:00:00') & (df['TimeStemp'] <= '2016-04-30 23:59:59') & (df.index % 2 == 0)]
print(df_3d.shape)

(1740, 245)


## 1.2. Filter columns

We'll get only the MEAN columns

In [37]:
df_3dmean = df_3d.filter(regex='(Orientation|Pressure|Rotation).*_MEAN$', axis=1)
list(df_3dmean)



['Pressure_MEAN',
 'OrientationProbe_azimuth_MEAN',
 'OrientationProbe_pitch_MEAN',
 'OrientationProbe_roll_MEAN',
 'RotationVector_cosThetaOver2_MEAN',
 'RotationVector_xSinThetaOver2_MEAN',
 'RotationVector_ySinThetaOver2_MEAN',
 'RotationVector_zSinThetaOver2_MEAN']

RotationVector_cosThetaOver2_MEAN is a feature with all values as NaN so we'll exclude it.

In [38]:
 df_3dmean = df_3dmean.loc[:, df_3dmean.columns.difference(["RotationVector_cosThetaOver2_MEAN"])]

Drop null values too

In [39]:
df_3dmean = df_3dmean.dropna()
print(df_3dmean.shape)

(1740, 7)


# 2. Principal Component Analysis
## 2.1. Scalation

In [8]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
datanorm = scaler.fit_transform(df28mean)

ValueError: Found array with 0 feature(s) (shape=(1740, 0)) while a minimum of 1 is required by StandardScaler.

## 2.2. Modelling (PCA)

In [ ]:
from sklearn.decomposition import PCA

n_components = 3
estimator = PCA(n_components)
X_pca = estimator.fit_transform(datanorm)

## 2.3. Plotting (PCA)

In [ ]:
import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = Axes3D(fig)
x = X_pca[:,0]
y = X_pca[:,1]
z = X_pca[:,2]
ax.scatter(x,y,z)
plt.show()

# 3. Clustering

## 3.1. Using KMeans

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics

iterations = 10
max_iter = 300
tol = 1e-04 
random_state = 0
k = 132
init = "random"
km = KMeans(k, init, n_init = iterations ,max_iter= max_iter, tol = tol,random_state = random_state)
labels = km.fit_predict(df28mean)

print(metrics.silhouette_score(df28mean, labels))

In [ ]:
x = X_pca[:,0]
y = X_pca[:,1]
plt.scatter(x,y, c = labels)

## 3.2. Plotting centroids

In [ ]:
plt.scatter(km.cluster_centers_[:,0], km.cluster_centers_[:,1], c='red',s=50)
plt.show()

In [ ]:
fig = plt.figure()
ax = Axes3D(fig)
x = X_pca[:,0]
y = X_pca[:,1]
z = X_pca[:,2]
ax.scatter(km.cluster_centers_[:,0], km.cluster_centers_[:,1], km.cluster_centers_[:,2], c='red',s=50)
ax.scatter(x,y,z, c = labels)
plt.show()